In [5]:
import pandas as pd
import numpy as np

In [6]:
msk = pd.read_csv('../realises2/data.csv')

In [7]:
msk.head(1)

,id,number_of_rooms,metro,adress,apartment_area,house,parking,price_per_month,repair,balcony,windows,bathroom,child_or_animals,ceiling_height,elevator,garbage_chute
0,271271157,4,Рядом,"Москва, улица Новый Арбат, 27",200,"5/16, Монолитный",подземная,500000,Дизайнерский,"Балкон (0), Лоджия (0)",Во двор,Совмещенный (1),"Можно с детьми, Можно с животными",3.0,"Пасс (4), Груз (1)",Да


In [23]:
msk.drop(['adress'], axis=1, inplace=True)

In [ ]:
msk['house_type'] = msk['house'].str.extract(r',\s*([\w\s]+)$')

In [24]:
def floors(val):
    part = val.split(',')
    if len(part) > 0:
        floor = part[0].strip()
        if '/' in floor:
            return floor
    return 0

msk['floor'] = msk['house'].apply(floors)

In [27]:
msk.drop('house', axis=1, inplace=True)

In [29]:
msk['house_type'].value_counts()

house_type
Панельный      6679
Кирпичный      3696
Монолитный     3615
Блочный        1689
Сталинский      141
старый фонд      68
Деревянный        5
Щитовой           1
Name: count, dtype: int64

In [28]:
msk['floor'].value_counts()

floor
3/5      524
4/5      511
2/5      481
5/5      433
5/9      393
        ... 
10/29      1
19/29      1
7/29       1
41/60      1
25/42      1
Name: count, Length: 884, dtype: int64

In [30]:
#преобразование floors к нормальному распределению. Чем центрее - тем лучше
def floors_to_norma(val):
    try:
        curr, vsego = map(int, val.split('/'))
        mid = vsego // 2

        if abs(curr - mid) <= 1:
            return 1
        elif curr == 1 or curr == vsego:
            return 0.2
        else:
            diff = abs(curr - mid)
            return max(0.2, 1 - (diff/mid))
    except ValueError:
        return 0
    
msk['floor'] = msk['floor'].apply(floors_to_norma)

In [31]:
house_type = {
    'Панельный': 10,
    'Кирпичный': 12,
    'Монолитный': 9,
    'Блочный': 8,
    'Сталинский': 6,
    'старый фонд': 2, 
    'Деревянный': 1,
    'Щитовой': 1
}

msk['house_type'] = msk['house_type'].map(house_type)

In [32]:
msk['elevator'].value_counts()

elevator
Пасс (1)               9057
Пасс (2)               3997
Пасс (1), Груз (1)     3431
Пасс (2), Груз (1)     1091
Пасс (2), Груз (2)      606
Пасс (3)                545
Пасс (4)                307
Пасс (3), Груз (1)      170
Пасс (1), Груз (2)      168
Пасс (4), Груз (2)       60
Груз (1)                 57
Пасс (4), Груз (1)       55
Пасс (3), Груз (3)       42
Пасс (3), Груз (2)       36
Груз (4)                 25
Пасс (4), Груз (4)       23
Груз (2)                 18
Груз (3)                 15
Пасс (4), Груз (3)        7
Пасс (6)                  6
Пасс (1), Груз (3)        5
Пасс (60)                 2
Пасс (2), Груз (3)        2
Пасс (50)                 2
Пасс (5), Груз (1)        1
Пасс (1), Груз (12)       1
Пасс (8), Груз (8)        1
Пасс (7)                  1
Пасс (1), Груз (4)        1
Пасс (5), Груз (3)        1
Пасс (2), Груз (4)        1
Груз (6)                  1
Груз (8)                  1
Пасс (5)                  1
Name: count, dtype: int64

In [38]:
#Функция для пассажирских
def passanger(val):
    if val:
        parts = [part for part in val.replace(',','').split() if part.startswith('Пасс')]
        if parts:
            try:
                return int(parts[0].split('(')[1].strip(')'))
            except:
                return 0
    return 0

    return 0

def gruz(val):
    if val:
        parts = [part for part in val.replace(',','').split() if part.startswith('Груз')]
        if parts:
            try:
                return int(parts[0].split('(')[1].strip(')'))
            except:
                return 0
    return 0

msk['elevator_pass'] = msk['elevator'].apply(passanger)
msk['elevator_gruz'] = msk['elevator'].apply(gruz)


In [39]:
msk.drop('elevator', axis=1, inplace=True)

In [40]:
msk.head(1)

,id,number_of_rooms,metro,apartment_area,parking,price_per_month,repair,balcony,windows,bathroom,child_or_animals,ceiling_height,garbage_chute,floors,house_type,floor,elevator_pass,elevator_gruz
0,271271157,4,Рядом,200,подземная,500000,Дизайнерский,"Балкон (0), Лоджия (0)",Во двор,Совмещенный (1),"Можно с детьми, Можно с животными",3.0,Да,True,9.0,0.625,0,0


In [41]:
msk.drop('floors', axis=1, inplace=True)